In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
!pip install Augmentor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.layers import Input, Lambda, Conv2D, MaxPool2D ,Dense, Flatten, Dropout
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
model = VGG19()
model.summary()

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/FYP/DataSet/Trainning'
valid_path = '/content/drive/MyDrive/FYP/DataSet/Validation'
test_path = '/content/drive/MyDrive/FYP/DataSet/Test'


# add preprocessing layer to the front of VGG
# sc: use resnet, solve problem for vanishing gradient
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(vgg.output)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
prediction = Dense(2, activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)




In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical')

validation_set = test_datagen.flow_from_directory(valid_path,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')


In [ ]:
from os import listdir

label = listdir(train_path)
numClass = len(label)
print (label)

In [ ]:
r = model.fit(training_set, epochs=20, validation_data = validation_set, validation_steps = 1)

In [ ]:
model.save("/content/drive/MyDrive/FYP/modelV2_VGG19_8to2_ep20.h5")

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# loss
plt.figure(1)
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.title("train-val loss graph")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()
plt.savefig("/content/drive/MyDrive/FYP/Result")

# accuracies
plt.figure(2)
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.title("train-val accuracy graph")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()
plt.savefig("/content/drive/MyDrive/FYP/Result")

In [ ]:
import numpy as np
train_imgs, train_labels = next(training_set)
# ensemble predictions and submit
train_results = np.zeros( (1400,2) )

train_results = model.predict(train_imgs, verbose = 1)
train_results = np.argmax(train_results, axis = -1)
print(train_results)
train_labels = np.argmax(train_labels,axis = -1)
train_labels
print(train_labels)
# Performance Evaluation
FP = len(np.where(train_results - train_labels == 1)[0])
FN = len(np.where(train_results - train_labels == -1)[0])
TP = len(np.where(train_results + train_labels == 0)[0])
TN = len(np.where(train_results + train_labels == 2)[0])
cmat = [[TP, FP], [FN, TN]]
total = TP + FP + FN + TN
accuracy = (TP + TN) / total * 100
sensitivity = TP / (TP + FN) * 100
specificity = TN / (TN + FP) * 100
precision = TP / (TP + FP) * 100
f1_score = (2 * precision * sensitivity) / (precision + sensitivity)
print('Training accuracy is: {0:0.2f}%'.format(accuracy))
print('Training sensitivity is: {0:0.2f}%'.format(sensitivity))
print('Training specificity is: {0:0.2f}%'.format(specificity))
print('Training precision is: {0:0.2f}%'.format(precision))
print('Training F1-score: {0:0.2f}'.format(f1_score))